In [ ]:
import networkx as nx
import sys
import os
from termcolor import colored
from networkx.algorithms.centrality import betweenness_centrality, degree_centrality, closeness_centrality, eigenvector_centrality

In [ ]:
methods_list = ['K-Shell', 'CIM', 'CBIM', 'MIM Reasoner', 'MA IMMULTI', 'Our Method']
robustness_centrality_list = {'b':betweenness_centrality, 'd':degree_centrality,
                              'c':closeness_centrality, 'e':eigenvector_centrality}
robustness_centrality_list_str = {'b':'betweenness', 'd':'degree',
                              'c':'closeness', 'e':'eigenvector'}
robustness_centrality = robustness_centrality_list['d']
robustness_centrality_str = robustness_centrality_list_str['d']
robustness_beta = 0.5
draw_network = False

In [ ]:
ordered_color_list = ['green', 'blue', 'teal', 'purple', 'orange', 'red', 'brown', 'pink', 'gray', 'black', 'gold', 'yellow', 'cyan', 'lime', 'magenta']
ordered_marker_list = ["o", "d", "s", "*", 'P', "2", 's', "v", "X", "8", ".", '^', "<", "<", "+", ]
plt_color_list_dict , plt_marker_list_dict = {}, {}
for i, item in enumerate(methods_list):
    plt_color_list_dict[item] = ordered_color_list[i]
    plt_marker_list_dict[item] = ordered_marker_list[i]

print(colored(plt_color_list_dict, 'yellow'), '\n', colored(plt_marker_list_dict, 'green'))

# plt_color_list_dict = ['green', 'cyan', 'blue', 'red', 'orange', 'black', 'brown', 'gold', 'yellow', 'magenta',
#                         'lime', 'pink', 'gray', 'purple', 'silver']
# plt_marker_list_dict = ["o", "d", "s", "*", 'P', "2", "p", "v",
#                         "X", "8", ".", '^', "<", "<", "+", ]

In [ ]:
CLASSES_PATH = os.path.dirname(os.path.abspath('D:/Masters thesis/Code/Classes/'))
if not (CLASSES_PATH in sys.path):
    sys.path.append(CLASSES_PATH)
from Classes.Files_Handler_Class import Files_Handler
from Classes.Resd_Network_Infos_Class import Resd_Network_Infos


In [ ]:
color_list = ["light_red", "light_green", "light_yellow",
               "light_blue","light_magenta", "light_cyan",
               "blue", "red", "white", "green", "yellow",
                 "magenta", "cyan", ]

In [ ]:
files_handler_obj = Files_Handler()
file_path = files_handler_obj.select_files("text files", ".edgeslist .edgelist .edges .mtx .txt")
if file_path is None or file_path == '':
    sys.exit("File Selection Canceled !")
file_info = files_handler_obj.get_file_path_info(file_path)
network_name = file_info['name']
network_type = file_info['type']
network_path = file_info['path']
if network_name == "":
    sys.exit("Dont Network Selection!")
file_info


In [ ]:
compares_path = files_handler_obj.make_dir(file_info['path'], f'Compare_Methods_{robustness_centrality_str}')

In [ ]:
resd_network_infos_object = Resd_Network_Infos()
(
    network_layers_info,
    network_layers_nodes,
    entra_layer_edges,
    entra_layer_edges_features,
    inter_layer_edge,
) = Resd_Network_Infos.read_nodeFrom_layerFrom_nodeTo_layerTo(
    network_path, network_name, network_type
)


In [ ]:
print(file_info['name'], '\n')
sum_of_layers_nodes = 0
network_layers_count = len(network_layers_info)
graphs_of_network = [None] * network_layers_count
network_entier_edges = ""
layers_nodes_infect_scale = []

i = 0
j = 0
while i < network_layers_count:
    graphs_of_network[i] = nx.Graph()
    network_layers_nodes[i] = list(set(network_layers_nodes[i]))
    layers_nodes_infect_scale.append({})
    if len(network_layers_nodes[i]) > 0:
        # graphs_of_network[i].add_nodes_from(network_layers_nodes[i])
        graphs_of_network[i].add_edges_from(entra_layer_edges[i])
        nx.set_node_attributes(graphs_of_network[i], None, 'community')

        graphs_of_network[i].graph["id"] = i
        graphs_of_network[i].graph["node_num"] = graphs_of_network[i].number_of_nodes()
        graphs_of_network[i].graph["edge_num"] = graphs_of_network[i].number_of_edges()

        print(colored("Layer "  + str(i) + ": " + str(graphs_of_network[i].number_of_nodes()) + " Node And " +
                       str(graphs_of_network[i].number_of_edges()) + " Edge", color_list[j]))
        # print(colored(graphs_of_network[i].graph['k_shell_info'], color_list[i]))
    i += 1
    j += 1
    if j >= len(color_list):
        j = 0

network_entier_nodes_list = []
for item in network_layers_nodes:
    network_entier_nodes_list += item
    sum_of_layers_nodes += len(item)

network_entier_nodes_list = list(set(network_entier_nodes_list))
network_entier_nodes_count = len(network_entier_nodes_list)
print()
print("network entier nodes : " + colored(str(network_entier_nodes_count), "yellow"))

In [ ]:
import json
import re
import math
import os
import networkx as nx
from networkx.algorithms.centrality import betweenness_centrality, degree_centrality, closeness_centrality, eigenvector_centrality

def load_results_from_file(root_path:os.path, method_title):
    files_handler_obj = Files_Handler()
    root_path = root_path + method_title + '/'
    files_list = files_handler_obj.get_files_in_path(root_path)
    # print(files_list)
    infection_result_files = []
    regex1 = r"^infection k="
    regex2 = r".json$"
    for item in files_list:
        if re.findall(regex1, item) and re.findall(regex2, item):
            infection_result_files.append(item)

    info = {}
    load_flag = False
    for item in infection_result_files:
        load_flag = False
        k = item.split(" ")[1].split("=")[-1]
        try:
            with open(root_path + item, 'r') as openfile:
                info[k] = json.load(openfile)
            load_flag = True
        except Exception as e:
            load_flag = False
            print(k, e)       

    if len(info.keys()) <= 0:
        info = None
    return info, method_title, load_flag

def get_seedset_degree_list(graphs_of_network, seedset):
    node_degree_list = []
    sum_of_degrees = 0
    node_existence_counter = 0
    layer_counter = 0
    for node in seedset:
        node_degree_list.append([])
        layer_counter = 0
        for j, graph in enumerate(graphs_of_network):
            if graph.number_of_nodes() > 0 and node in graph:
                layer_counter += 1
                node_degree = graph.degree(node)
                sum_of_degrees += node_degree
                node_existence_counter += 1
                node_degree_list[-1].append(node_degree)
            elif graph.number_of_nodes() > 0:
                layer_counter += 1
    degree_avg = sum_of_degrees / node_existence_counter
    layer_participation = node_existence_counter / (len(seedset) * layer_counter)
    return node_degree_list, degree_avg, layer_participation

def get_distance_seedset_nodes_each_other(graphs_of_network, seedset):
    sum_of_distance = 0
    counter = 0
    
    if len(seedset) < 2:
        return 0

    for graph in graphs_of_network:
        i = 0
        while i < len(seedset):
            j = i + 1
            while j < len(seedset):
                if graph.number_of_edges() > 0:
                    if seedset[i] in graph and seedset[j] in graph:
                        try:
                            dist = nx.shortest_path_length(graph, source=seedset[i], target=seedset[j])
                        except:
                            dist = 6
                        sum_of_distance += dist
                        counter += 1
                    else:
                        sum_of_distance += 6
                        counter += 1
                j += 1
            i += 1

    return (sum_of_distance/counter)

def get_seedset_similarity(r1, r2):
    similarity = 0
    for item in r1 :
        if item in r2:
            similarity += 1
    return (similarity / len(r1))

def evaluate_robust_influence(graph:nx.Graph, seed_nodes:list, beta:float=1.5,
                              robustness_centrality:nx.algorithms.centrality=nx.algorithms.centrality.betweenness_centrality):
    """
    Evaluate both:
        - RS-cf: Robust influence score for the given seed nodes
        - sigma_cf_ang: Structural influence score for seedset in the network

    Parameters:
        graph (networkx.Graph): The input graph (undirected or directed)
        seed_nodes (list): List of initial seed node IDs
        beta (float): Tolerance parameter used for capacity calculation

    Returns:
        rs_cf (float): RS-cf value for the seed nodes
        sigma_cf_dict (dict): A dictionary {node: sigma_cf value}
    """
    G = graph.copy()
    t = 0  # Number of attack rounds
    sigma_values = []  # Influence scores after each attack (for RS-cf)
    influence_track = {i: [] for i in G.nodes}  # Influence history per node
    survival_time = {}  # Survival time under cascading failure

    while len(G) > 0:
        # Step 1: Compute load and capacity for all nodes
        load = robustness_centrality(G)
        if not load:
            break
        capacity = {i: beta * load[i] for i in G.nodes}

        # Step 2: Remove the node with the highest load
        target = max(load, key=load.get)
        G.remove_node(target)
        t += 1
        survival_time[target] = t

        # Step 3: Cascading removals based on overloaded nodes
        while True:
            if len(G) > 0:
                load = robustness_centrality(G)
                failed = [i for i in G.nodes if load.get(i, 0) > capacity.get(i, float('inf'))]
                if not failed:
                    break
                for node in failed:
                    survival_time[node] = t
                G.remove_nodes_from(failed)
            else:
                break

        # Step 4: RS-cf - compute total reachable nodes from seed set
        reachable = set()
        for seed in seed_nodes:
            if seed in G:
                if G.is_directed():
                    reachable |= nx.descendants(G, seed)
                else:
                    reachable |= nx.node_connected_component(G, seed)
        sigma_values.append(len(reachable))

        # Step 5: sigma_cf - track influence for each node
        for node in G.nodes:
            if G.is_directed():
                influence_track[node].append(len(nx.descendants(G, node)))
            else:
                influence_track[node].append(len(nx.node_connected_component(G, node)))

    # Assign remaining nodes (if any) a survival time of t+1
    for node in G.nodes:
        survival_time[node] = t + 1

    # Final σ̂_cf computation
    sigma_cf_dict = {}
    for i in influence_track:
        rank_i = survival_time.get(i, t + 1)
        sigma_cf_dict[i] = sum(influence_track[i][:rank_i])

    # Final RS-cf computation
    rs_cf = sum(sigma_values) / t if t > 0 else 0

    # Calculate average sigma_cf for seed nodes
    sigma_cf_sum = 0
    for seed_node in seed_nodes:
        if seed_node in sigma_cf_dict:
            sigma_cf_sum += sigma_cf_dict[seed_node]
    sigma_cf_avg = sigma_cf_sum / len(seed_nodes) if len(seed_nodes) > 0 else 0

    return rs_cf, sigma_cf_avg

def get_evaluate_robust_influence(graphs_of_network:list[nx.Graph], seed_nodes:list, beta:float=1.5,
                                   robustness_centrality:nx.algorithms.centrality=nx.algorithms.centrality.betweenness_centrality):
    rs_cf,  sigma_cf_avg = 0, 0
    layer_counter = 0
    for graph in graphs_of_network:
        if graph.number_of_nodes() > 0:
            temp_rs_cf, temp_sigma_cf_avg = evaluate_robust_influence(graph, seed_nodes, beta, robustness_centrality)
            rs_cf += temp_rs_cf
            sigma_cf_avg += temp_sigma_cf_avg
    #         layer_counter += 1
    # if layer_counter > 0:
    #     rs_cf /= layer_counter
    #     sigma_cf_avg /= layer_counter
    return rs_cf, sigma_cf_avg

def results_annalise (graphs_of_network, results, other_results, bete:float=None, robustness_centrality=None):
    results_annalise_info = {}
    if not(results is None):
        for k, v in results.items():
            results_annalise_info[k] = {}
            (
                results_annalise_info[k]["seedset_degree_list"],
                results_annalise_info[k]["seedset_degree_average"],
                results_annalise_info[k]["seedset_layer_participation"],
            ) = get_seedset_degree_list(graphs_of_network, v['seed_set'])

            
            (
                results_annalise_info[k]["rs_cf"],
                results_annalise_info[k]["sigma_cf_avg"]
            ) = get_evaluate_robust_influence(graphs_of_network, v['seed_set'], beta=bete, robustness_centrality=robustness_centrality)
            
            similarity = 0
            temp_similarity = 0
            for result in other_results:
                if not (result is None):
                    temp_similarity += get_seedset_similarity(v['seed_set'], result[k]['seed_set'])
            similarity += (temp_similarity / len(other_results))

            results_annalise_info[k]["seedset_similarity"] = similarity

            results_annalise_info[k]["seedset_distance_average"] = get_distance_seedset_nodes_each_other(graphs_of_network, v['seed_set'])
    return results_annalise_info



In [ ]:
methods_results = {}
for item in methods_list:
    method_results, method_title, method_results_flag = load_results_from_file(file_info['path'], item)
    if method_results_flag:
        methods_results[item] = {}
        methods_results[item]['results'] = method_results
        methods_results[item]['title'] = method_title
        methods_results[item]['status'] = method_results_flag

In [ ]:
i = 1
for method, results in methods_results.items():
    other_method_results = []
    print(str(i)+"- ", method)
    for t_method, t_results in methods_results.items():
        if t_method != method:
            other_method_results.append(t_results['results'])
    i += 1
    if len(other_method_results) > 0:
        methods_results[method]['results_annalise'] = results_annalise(graphs_of_network, results['results'], other_method_results,
                                                                       robustness_beta, robustness_centrality)

In [ ]:
k_values = list(methods_results[methods_list[0]]['results'].keys())
k_values = [int(k) for k in k_values]
k_values = sorted(k_values)
k_values

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import numpy as np

def plot_line(plt, infection_scale, k_values, method_results, label, color, marker):
    results_infection = []
    for k in k_values:
        results_infection.append(method_results[str(k)][infection_scale])
    plt.plot(k_values, results_infection, label=label, linestyle='solid', marker=marker,  color=color)

def plot_lines(plt,compartion_parameter, infection_scale, k_values, method_results,
                method_results_annalise, color, marker, visable_infection:bool=False):
    results_degree_average = []
    results_infection = []
    for k in k_values:
        results_degree_average.append(method_results_annalise[str(k)][compartion_parameter])
        results_infection.append(method_results[str(k)][infection_scale])
    plt.plot(k_values, results_degree_average, linestyle='dashed', marker=marker,  color=color)
    if visable_infection:
        plt.plot(k_values, results_infection, linestyle='solid', marker=marker,  color=color)

def plot_comparison(compartion_parameter,infection_scale, k_values,
                    compartion_label, infection_label,
                    x_axis_lable, diagram_title,
                    list_of_results,
                    list_of_titles,
                    list_of_results_annalise,
                    plt_color_list,
                    plt_marker_list,
                    save_result_path,
                    legend_loc:str='best', legend_size='small', legend_framealpha = 0.5,
                    visable_infection:bool=False):
    # fig = plt.figure()
    fig, ax1 = plt.subplots()
    ax1.set_xticks(k_values)
    handles, labels = plt.gca().get_legend_handles_labels()
    i = 0
    for j, results in enumerate(list_of_results):
        if i >= len(plt_color_list) - 1:
            i = 0
        plot_lines(plt,compartion_parameter,infection_scale, k_values, results,
                    list_of_results_annalise[j], plt_color_list[list_of_titles[j]],plt_marker_list[list_of_titles[j]], visable_infection)
        handles.append(Line2D([0], [0], label=list_of_titles[j], color=plt_color_list[list_of_titles[j]], linestyle='', marker=plt_marker_list[list_of_titles[j]]))
        i += 1

    if visable_infection:
        handles.append(Line2D([0], [0], label=infection_label, color='black', linestyle='solid'))
        handles.append(Line2D([0], [0], label=compartion_label, color='black', linestyle='dashed'))

    plt.rcParams['font.family'] = 'serif'  # Or 'sans-serif', 'monospace', etc.
    plt.rcParams['font.serif'] = ['Times New Roman']  # Try other fonts too
    
    plt.xlabel(x_axis_lable, fontsize=16)
    plt.ylabel(compartion_label, fontsize=16)
    # plt.title(diagram_title)
    plt.legend(handles=handles, loc=legend_loc, fontsize=legend_size, framealpha=legend_framealpha)

    # Tick labels
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    # plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    # plt.margins(0,0)
    # plt.gca().xaxis.set_major_locator(plt.NullLocator())
    # plt.gca().yaxis.set_major_locator(plt.NullLocator())
    
    plt.show()
    fig.savefig(save_result_path + diagram_title + '.png', bbox_inches='tight', pad_inches = 0.05)

def plot_infection_comparison(infection_scale, k_values,
                              infection_label, x_axis_lable, diagram_title,
                              list_of_results, list_of_titles,
                              plt_color_list, plt_marker_list,
                              save_result_path,
                              legend_loc='best',
                              legend_size=14, legend_framealpha=0.5):
    # fig = plt.figure()
    fig, ax1 = plt.subplots()
    ax1.set_xticks(k_values)

    
    
    for j, results in enumerate(list_of_results):
        plot_line(plt,infection_scale, k_values, results, list_of_titles[j],
                  plt_color_list[list_of_titles[j]], plt_marker_list[list_of_titles[j]])

    plt.rcParams['font.family'] = 'serif'  # Or 'sans-serif', 'monospace', etc.
    plt.rcParams['font.serif'] = ['Times New Roman']  # Try other fonts too
    
    # plt.subplots_adjust(bottom=.2, left=.2)
    plt.xlabel(x_axis_lable, fontsize=16)
    plt.ylabel(infection_label, fontsize=16)
    # plt.title(diagram_title, fontsize=20)
    plt.legend(loc=legend_loc, fontsize=legend_size, framealpha=legend_framealpha)

    # Tick labels
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    # plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    # plt.margins(0,0)
    # plt.gca().xaxis.set_major_locator(plt.NullLocator())
    # plt.gca().yaxis.set_major_locator(plt.NullLocator())

    plt.show()
    fig.savefig(save_result_path + diagram_title + '.png', bbox_inches='tight', pad_inches = 0.05)

def plot_lines_and_bars(
    x,    # List of x-values (e.g., k values)
    line_series:dict,    # List of dicts: [{'y': [...], 'label': 'Line 1', 'color': 'blue'}, ...]
    bar_series:dict,     # List of dicts: [{'y': [...], 'label': 'Bar 1', 'color': 'orange'}, ...]
    xlabel:str,
    ylabel_l:str,
    ylabel_r:str,
    rcParams:dict={'axes.titlesize': 16, 'axes.labelsize': 16,
                   'xtick.labelsize': 16, 'ytick.labelsize': 16,
                   'legend.loc':'upper left'},  # Optional rcParams for customization
    diagram_title:str='',
    save_result_path:str='',
):

    plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],  # or another font
    'axes.titlesize': rcParams.get('axes.titlesize', 16),
    'axes.labelsize': rcParams.get('axes.labelsize', 16),
    'xtick.labelsize': rcParams.get('xtick.labelsize', 16),
    'ytick.labelsize': rcParams.get('ytick.labelsize', 16),
})

    x = np.array(x)
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()

    bar_width = 0.8 / max(len(bar_series), 1)  # auto adjust bar width


    # Plot lines on top
    if line_series:
        for i, line in enumerate(line_series):
            ax1.plot(x, line['y'],
                     marker=line.get('marker', '.'),
                     color=line.get('color', 'black'),
                     label=line.get('label', f'Line {i+1}'),
                     zorder=2)


    # Plot bars first (behind lines)
    if bar_series:
        for i, bar in enumerate(bar_series):
            shift = (i - (len(bar_series) - 1) / 2) * bar_width
            ax2.bar(x + shift, bar['y'], width=bar_width,
                    color=bar.get('color', 'gray'),
                    alpha=0.3,
                    label=bar.get('label', f'Bar {i+1}'),
                    zorder=1)

    
    # Labels and ticks
    ax1.set_xlabel(xlabel, color='black')
    ax1.set_ylabel(ylabel_l, color='black')
    ax2.set_ylabel(ylabel_r, color='black')

    ax1.set_xticks(x)
    ax1.tick_params(axis='y', labelcolor='black')
    ax2.tick_params(axis='y', labelcolor='black')

    # Combine legends
    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    fig.legend(handles1 + handles2, labels1, loc=rcParams.get('legend.loc', 'upper left'),
               fontsize=rcParams.get('legend.fontsize', 14),
               bbox_to_anchor=rcParams.get('legend.bbox_to_anchor', None),
               framealpha = rcParams.get('legend.framealpha', 0.5)
               )

    # plt.title("Multi Line and Bar Chart")
    fig.tight_layout()
    plt.show()
    fig.savefig(save_result_path + diagram_title + '.png', bbox_inches='tight', pad_inches = 0.05)

def plot_infection_comparison_line_bar(k_values:list, list_of_results:dict,
                                       list_of_titles:dict, plt_color_list_dict:dict,
                                       plt_marker_list_dict:dict, compares_path:str,
                                       rcParams:dict={'axes.titlesize': 16, 'axes.labelsize': 16,
                                                        'xtick.labelsize': 16, 'ytick.labelsize': 16,
                                                        'legend.loc':'upper left'}):
    line_series = []
    bar_series = []
    for j, results in enumerate(list_of_results):
        line_series.append({})
        bar_series.append({})

        line_series[-1]['y'] = []
        bar_series[-1]['y'] = []
        
        bar_series[-1]['label'] = list_of_titles[j]
        line_series[-1]['label'] = list_of_titles[j]

        line_series[-1]['color'] = plt_color_list_dict[list_of_titles[j]]
        bar_series[-1]['color'] = plt_color_list_dict[list_of_titles[j]]

        line_series[-1]['marker'] = plt_marker_list_dict[list_of_titles[j]]
        bar_series[-1]['marker'] = plt_marker_list_dict[list_of_titles[j]]

        for k in k_values:
            line_series[-1]['y'].append(results[str(k)]['infection'])
            bar_series[-1]['y'].append(results[str(k)]['percentage'])


    plot_lines_and_bars(
        x=k_values,
        line_series=line_series,
        bar_series=bar_series,
        xlabel='Seedset Size (k)',
        ylabel_l='Infection Node Count',
        ylabel_r='Percentage of Infected Nodes',
        rcParams = rcParams,
        diagram_title='Infection_Percentage_Comparison',
        save_result_path=compares_path
    )
        

    pass



In [ ]:
list_of_results = []
list_of_titles = []
list_of_results_annalise = []
for method, results in methods_results.items():
    list_of_results.append(results['results'])    
    list_of_titles.append(results['title'])
    list_of_results_annalise.append(results['results_annalise'])

file_info['name']

In [ ]:
rcParams={'axes.titlesize': 16, 'axes.labelsize': 16,
          'xtick.labelsize': 16, 'ytick.labelsize': 16,
          'legend.fontsize':12,

          'legend.loc':'upper left',
          'legend.bbox_to_anchor':(0.135, 0.975),
          # 'legend.loc':'lower right',
          # 'legend.bbox_to_anchor':(0.88, 0.13),

          'legend.framealpha':0.1}
plot_infection_comparison_line_bar(k_values, list_of_results, list_of_titles, plt_color_list_dict, plt_marker_list_dict, compares_path,
                                   rcParams=rcParams)

In [ ]:
plot_infection_comparison(
    'infection', k_values,
    'Infection', 'Seedset Size',
    'Infection Node Comparison',
    list_of_results,
    list_of_titles,
    plt_color_list_dict, plt_marker_list_dict,
    compares_path,
    'best',
    legend_framealpha=0.5,
)

In [ ]:
plot_infection_comparison(
    'percentage', k_values,
    'Infection', 'Seedset Size',
    'Infection Percentage Comparison',
    list_of_results,
    list_of_titles,
    plt_color_list_dict, plt_marker_list_dict,
    compares_path,
    'best',
    legend_framealpha=0.5,
)

In [ ]:
plot_comparison(
    "seedset_degree_average", 'infection', k_values,
    'Degree Average','Infection', 'Seedset Size',
    'Degree Average Comparison',
    list_of_results,
    list_of_titles,
    list_of_results_annalise,
    plt_color_list_dict, plt_marker_list_dict,
    compares_path,
    'best',
    # 'center left',
    # legend_size='small'
    legend_size=14,
    legend_framealpha=0.5,
)

In [ ]:
plot_comparison(
    "seedset_layer_participation", 'percentage', k_values,
    'Layer Participation','Infection Percentage','Seedset Size',
    'Layer Participation Comparison',
    list_of_results,
    list_of_titles,
    list_of_results_annalise,
    plt_color_list_dict, plt_marker_list_dict,
    compares_path,
    'best',
    # 'upper right',
    # 'lower right',
    # legend_size='small'
    legend_size=14,
    legend_framealpha=0.3
)

In [ ]:
plot_comparison(
    "seedset_distance_average", 'percentage', k_values,
    'Distance Average','Infection Percentage','Seedset Size',
    'Distance Average Comparison',
    list_of_results,
    list_of_titles,
    list_of_results_annalise,
    plt_color_list_dict, plt_marker_list_dict,
    compares_path,
    'best',
    legend_size=14,
    legend_framealpha=0.5,
)

In [ ]:
plot_comparison(
    "seedset_similarity", 'percentage', k_values,
    'Seedset Similarity','Infection', 'Seedset Size',
    'Seedset Similarity Comparison',
    list_of_results,
    list_of_titles,
    list_of_results_annalise,
    plt_color_list_dict, plt_marker_list_dict,
    compares_path,
    'best',
    # 'lower right',
    # 'center left',
    # legend_size='small'
    legend_size=14,
    legend_framealpha=0.3,
)

In [ ]:
plot_comparison(
    "rs_cf", 'percentage', k_values,
    'rs_cf','Infection', 'Seedset Size',
    'rs_cf Comparison',
    list_of_results,
    list_of_titles,
    list_of_results_annalise,
    plt_color_list_dict, plt_marker_list_dict,
    compares_path,
    'best',
    # 'upper left',
    # 'upper center',
    # legend_size='small'
    legend_size=14,
    legend_framealpha=0.3,
)

In [ ]:
plot_comparison(
    "sigma_cf_avg", 'percentage', k_values,
    'sigma_cf_avg','Infection', 'Seedset Size',
    'sigma_cf_avg Comparison',
    list_of_results,
    list_of_titles,
    list_of_results_annalise,
    plt_color_list_dict, plt_marker_list_dict,
    compares_path,
    'best',
    # 'upper center',
    # 'center left',
    # legend_size='small'
    legend_size=14
)

In [ ]:
import networkx as nx
import plotly.graph_objects as go
def colorize_seedset_on_network(graphs_of_network, seedset, save_path, fig_title):
    for graph in graphs_of_network:
        if graph.number_of_nodes() > 0:   
            # Step 2: Generate 2D node positions
            pos = nx.spring_layout(graph)  # Spring layout for 2D positions

            # Extract node positions
            x_nodes = [pos[i][0] for i in graph.nodes()]  # x-coordinates of nodes
            y_nodes = [pos[i][1] for i in graph.nodes()]  # y-coordinates of nodes

            # Generate edge positions
            x_edges = []
            y_edges = []
            for edge in graph.edges():
                x_edges.extend([pos[edge[0]][0], pos[edge[1]][0], None])  # x-coordinates
                y_edges.extend([pos[edge[0]][1], pos[edge[1]][1], None])  # y-coordinates
            
            node_colors = []
            for node in graph.nodes():
                if node in seedset:
                    node_colors.append('red')
                else:
                    node_colors.append('yellow')

            # Step 3: Create Plotly figure
            fig = go.Figure()

            # Add edges
            fig.add_trace(go.Scatter(
                x=x_edges,
                y=y_edges,
                mode='lines',
                line=dict(color='blue', width=1),
                hoverinfo='none'
            ))

            # Add nodes
            fig.add_trace(go.Scatter(
                x=x_nodes,
                y=y_nodes,
                mode='markers+text',
                marker=dict(symbol='circle',
                            size=20, 
                            color=node_colors # Use the custom color list
                                ),
                text=[str(node) for node in graph.nodes()],  # Node labels
                textposition="middle center",
                hoverinfo='text',
            ))

            # Step 4: Customize layout
            fig.update_layout(
                title=fig_title + f" layer {graph.graph['id']}",
                showlegend=False,
                margin=dict(l=0, r=0, t=50, b=0),
                xaxis=dict(showgrid=False, zeroline=False),
                yaxis=dict(showgrid=False, zeroline=False),
                plot_bgcolor='white',
            )

            # Show the figure
            fig.show()
            fig.write_image(save_path + fig_title + f" layer {graph.graph['id']}.png")


In [ ]:
try:
    for k, v in methods_results[methods_list[-1]]['results'].items():
        print(v['seed_set'])
except:
    draw_network = False

In [ ]:
if draw_network:
    for k, v in methods_results[methods_list[-1]]['results'].items():
        colorize_seedset_on_network(graphs_of_network, v['seed_set'], compares_path, f"method {methods_results[methods_list[-1]]['title']} seedset size {k}")